### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, WESAD ####

In [34]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [35]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE,
# ]
] + train.Metrics.STATISTICAL

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

anxiety_label_type = None
wesad_label_type = "stai"

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40],
        "eval_metric": ["error", "log_loss"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    "random": None
}

threshold = "fixed"
test_size = 1.0

percent_of_target_dataset = 0.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [36]:
# TRAIN ON APD AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(73)

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_apd_wesad = {}
ensemble_weights_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_apd_wesad[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_apd_wesad, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_apd_wesad, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_apd_wesad, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_apd[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.57197801 0.559882   0.55530989 0.55828725 0.56646363 0.5512134
 0.55712377 0.55801002 0.55588625 0.55930135 0.5397652  0.52980418
 0.54881196 0.5469519  0.54759366 0.55398035 0.54322927 0.56075863
 0.55865651 0.56481853 0.54832815 0.55850743 0.55544831 0.55492893
 0.56379839        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.53148823 0.53086893 0.52946862 0.53150272 0.50188377 0.51885867
 0.51495019 0.51561953 0.51122654 0.51158522 0.51308203 0.51014065
 0.51835696 0.51609409 0.51517378 0.50889396 0.5261343  0.53897251
 0.53450318 0.53815156 0.52841124 0.53036791 0.53072452 0.52693954
 0.52490303 0.52734881 0.52921888 0.53024595 0.52212072 0.52854528
 0.53018817 0.5354274  0.54263383 0.54197677 0.53623503 0.52788738
 0.53859364 0.52508414 0.53333819 0.52460673 0.52100038 0.52140371
 0.53029684 0.53692274 0.55001789 0.54796801 0.54177244 0.54132578
 0.52579939 0.51725539 0.51486311 0.52648671 0.54969885 0.5334785
 0.54124503 0.53327393 0.52529589 0.5369837  0.54127022 0.54646219
 0.53473606 0.5317334  0.53111254 0.52964401        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan     

SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [428  95], Predictions: [0 1], [155 368]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [428  95], Predictions: [0 1], [144 379]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [428  95], Predictions: [0 1], [123 400]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 

In [37]:
# TRAIN ON WESAD AND TEST ON APD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(73)

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# print(n_wesad / n_apd)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_apd = {}
ensemble_weights_wesad = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_wesad_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_wesad[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.90402648 0.92513023 0.92468036 0.92325862 0.92079444 0.90300549
 0.92226225 0.92055633 0.917642   0.91694783 0.90304193 0.91461275
 0.91384027 0.91118688 0.91586708 0.88692359 0.90510736 0.91015403
 0.90754205 0.91142981 0.8853715  0.90591351 0.91260555 0.91080553
 0.91180155        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.82372616 0.82774548 0.83793631 0.89864245 0.82753188 0.82623001
 0.84131172 0.90273738 0.83141938 0.8294904  0.83905817 0.89923127
 0.83100785 0.8294904  0.83916105 0.8975646  0.8964974  0.88386864
 0.90128328 0.9113293  0.90607429 0.90648292 0.90168703 0.91103821
 0.90114807 0.89191903 0.89849719 0.90144729 0.91047925 0.90793846
 0.91389233 0.90718215 0.91126303 0.920764   0.92617096 0.93201377
 0.9145452  0.9270813  0.92857252 0.93387155 0.89687338 0.92714239
 0.93083434 0.93286804 0.90300177 0.92762351 0.93230844 0.9340651
 0.90096867 0.90550001 0.91266948 0.91508895 0.91806763 0.92573207
 0.92826062 0.92916311 0.91053377 0.91899095 0.92241427 0.92597419
 0.90663443 0.91255754 0.91899139 0.92122195        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan     

SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 6, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [570 462], Predictions: [0 1], [398 634]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [570 462], Predictions: [0 1], [680 352]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [570 462], Predictions: [0 1], [586 446]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'eda

In [38]:
# TRAIN ON WESAD AND TEST ON APD -- SPEECH TASK ONLY
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(73)

model_phases_apd_speech = [
    "Baseline_Rest", 
    # "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd_speech, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_wesad_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.91387315 0.93109789 0.93549906 0.9431634  0.94575949 0.92544619
 0.93075506 0.93506052 0.94011765 0.94188488 0.9251517  0.93180529
 0.93326828 0.9396194  0.93999485 0.92530273 0.92857981 0.93040105
 0.93506378 0.93672343 0.91994979 0.92470904 0.93180152 0.93364373
 0.93547312        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.85040668 0.85590064 0.85403418 0.86396216 0.85349698 0.86286855
 0.85971607 0.8654595  0.85093227 0.85659283 0.86062411 0.86629039
 0.85525326 0.85741587 0.86082987 0.86617857 0.89438329 0.92437822
 0.92878124 0.9297745  0.88503992 0.92037371 0.93129836 0.9247195
 0.88586744 0.91894883 0.92862646 0.92877537 0.89023987 0.91925748
 0.92790735 0.92790035 0.93461319 0.93028683 0.93086584 0.93711745
 0.93581899 0.92890035 0.92867386 0.93362824 0.92735251 0.93270085
 0.93567934 0.93637249 0.92606385 0.93270085 0.93567934 0.93688796
 0.93658132 0.93505988 0.9357351  0.93360035 0.93251246 0.932385
 0.93542754 0.93403402 0.92476165 0.92948775 0.9325311  0.93133614
 0.92902896 0.93222974 0.93756778 0.93469989        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan       

SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'binary_logloss', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [341 243], Predictions: [0 1], [269 315]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [341 243], Predictions: [0 1], [374 210]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [341 243], Predictions: [0 1], [385 199]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'e

In [39]:
# TRAIN ON WESAD AND TEST ON APD -- BUG TASK ONLY
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(73)

model_phases_apd_speech = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    # "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd_speech, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models_wesad_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.93611798 0.96187927 0.96301423 0.95742834 0.95999489 0.96251654
 0.96436775 0.96521022 0.95742183 0.95947783 0.9644444  0.96352038
 0.95904405 0.95180427 0.9540014  0.96429275 0.9636617  0.96154414
 0.95552464 0.95778194 0.95571487 0.96012513 0.95541523 0.95016509
 0.95529717        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.85489233 0.85335881 0.87983598 0.88098644 0.85847379 0.86038129
 0.88408883 0.88959323 0.87153559 0.87132247 0.8743276  0.88320278
 0.87153559 0.87132247 0.87369557 0.88425615 0.90457735 0.92203398
 0.92492215 0.93265505 0.91153318 0.91920009 0.91935609 0.927741
 0.90661745 0.91241517 0.91262926 0.92173833 0.90170172 0.91269607
 0.91698319 0.92145743 0.9183032  0.93520795 0.94006962 0.93797596
 0.92310502 0.92864109 0.93658633 0.9362875  0.91875108 0.92666644
 0.92996655 0.93029664 0.90891962 0.92175071 0.93024745 0.93240338
 0.93195946 0.94549487 0.94276214 0.94132722 0.9228908  0.9323595
 0.92834049 0.92961837 0.92024739 0.92349218 0.92301735 0.9189844
 0.93284082 0.93343478 0.93481327 0.93203094        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        

SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [297 255], Predictions: [0 1], [224 328]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [297 255], Predictions: [0 1], [375 177]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [297 255], Predictions: [0 1], [318 234]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'e

In [40]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier


random.seed(73)

percent_of_target_dataset = 0.0
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]


def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


In [41]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(73)

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

voting_type = "weighted_avg"
# voting_type = "majority_vote"

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_apd_wesad[name] for name in ensemble_models}
weights = [ensemble_weights_apd[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)

print("Train APD, test WESAD")
print(f"APD size: {x_train.shape[0]}\n{y_train.value_counts()}\nWESAD size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

###############################################################################################################

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_wesad = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_wesad / n_apd)) * n_wesad)
# augment_indices = random.sample(range(n_wesad), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_wesad_apd[name] for name in ensemble_models}
weights = [ensemble_weights_wesad[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print("Train WESAD, test APD")
print(f"WESAD size: {x_train.shape[0]}\n{y_train.value_counts()}\nAPD size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

Train APD, test WESAD
APD size: 1032
0    570
1    462
Name: label, dtype: int64
WESAD size: 523
0    428
1     95
Name: label, dtype: int64
Predictions:
[[  0 184]
 [  1 339]]
	Accuracy: 0.372848948374761
	Precision: 0.15634218289085547
	Recall: 0.5578947368421052
	F1-score: 0.24423963133640555
	AUC score: 0.4448352188883423
----------------------------------------
Train WESAD, test APD
WESAD size: 523
0    428
1     95
Name: label, dtype: int64
APD size: 1032
0    570
1    462
Name: label, dtype: int64
Predictions:
[[  0 671]
 [  1 361]]
	Accuracy: 0.5571705426356589
	Precision: 0.5069252077562327
	Recall: 0.3961038961038961
	F1-score: 0.44471445929526127
	AUC score: 0.541911597174755
----------------------------------------
